In [2]:
import tensorflow as tf
for device in tf.config.list_physical_devices():
    print(": {}".format(device.name))

: /physical_device:CPU:0
: /physical_device:GPU:0


In [3]:
# Import all necessary libraries
import os
import logging
import warnings
import numpy as np
import matplotlib.pyplot as plt

# Suppress warnings and logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.get_logger().setLevel(logging.ERROR)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# Set seeds for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Define constants
IMAGE_SIZE = (224, 224)  # Directly using ResNet50 input size
BATCH_SIZE = 64
TRAINING_DATA_PATH = "Final_Arabic_Alpha_dataset/train"
TEST_DATA_PATH = "Final_Arabic_Alpha_dataset/test"

# Load datasets
print("Loading datasets...")
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAINING_DATA_PATH,
    validation_split=0.1,
    subset="training",
    seed=SEED,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    label_mode='int'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    TRAINING_DATA_PATH,
    validation_split=0.1,
    subset="validation",
    seed=SEED,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    label_mode='int'
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DATA_PATH,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    label_mode=None,
    shuffle=False
)

# Get class names
class_names = train_ds.class_names
print(f"Class names: {class_names}")


AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

Loading datasets...
Found 42559 files belonging to 65 classes.
Using 38304 files for training.
Found 42559 files belonging to 65 classes.
Using 4255 files for validation.
Found 10640 files belonging to 1 classes.
Class names: ['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '7', '8', '9']


In [3]:
# Load the model
loaded_model = tf.keras.models.load_model("arabic_letter_classifier_95_87")

old_weights = loaded_model.layers[-1].get_weights()

loaded_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               524544

In [4]:
loss, accuracy = loaded_model.evaluate(train_ds)
val_loss, val_accuracy = loaded_model.evaluate(val_ds)

print(f"Training loss: {loss}, Training Accuracy: {accuracy}")
print(f"Val loss: {val_loss}, Val Accuracy: {val_accuracy}")

67/67 [==============================] - 12s 177ms/step - loss: 0.4037 - accuracy: 0.8717
Training loss: 0.17665907740592957, Training Accuracy: 0.9522504210472107
Val loss: 0.40373876690864563, Val Accuracy: 0.8716803789138794


In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Dense

# Identify the correct last dense layer before the output
x = loaded_model.layers[-2].output  # Target the last Dense layer before softmax
x = Dropout(0.2, name="new_dropout")(x)  # Add Dropout (rename it)
x = Dense(len(class_names), activation='softmax', name="new_output")(x)  # Rename new Dense layer

# Create new model (Keeps trained weights)
new_model = Model(inputs=loaded_model.input, outputs=x)

# Set the pre-trained weights back
new_model.layers[-1].set_weights(old_weights)  # Restore softmax layer weights

# Recompile and continue training
new_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 


 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 new_dropout (Dropout)       (None, 256)               0         
                                                                 
 new_output (Dense)          (None, 65)                16705     
                                                                 
Total params: 24,128,961
Trainable params: 541,249
Non-trainable params: 23,587,712
_________________________________________________________________


In [5]:
for layer in loaded_model.layers:
    if layer.name == 'dense':
        layer.traiable = True
    else:
        layer.trainable = False


base_model = new_model.get_layer('resnet50')

# Unfreeze some layers of the base model
fine_tune_at = 100  # Unfreeze from this layer onwards

base_model.trainable = True
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False


for layer in new_model.layers:
    print(f"{layer.name}: {layer.trainable}")
print()

new_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

new_model.summary()

input_2: False
tf.__operators__.getitem: False
tf.nn.bias_add: False
resnet50: True
global_average_pooling2d: False
dense: True
new_dropout: True
new_output: True

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)      

                                                                 
 new_output (Dense)          (None, 65)                16705     
                                                                 
Total params: 24,128,961
Trainable params: 19,994,177
Non-trainable params: 4,134,784
_________________________________________________________________


In [6]:
# Callbacks
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=2,
        min_lr=1e-8
    )
]

history = new_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,  # Continue training
    callbacks=callbacks
)

Epoch 1/5
599/599 [==============================] - 229s 330ms/step - loss: 0.8358 - accuracy: 0.7767 - val_loss: 0.3944 - val_accuracy: 0.8743 - lr: 1.0000e-05
Epoch 2/5
599/599 [==============================] - 183s 304ms/step - loss: 0.2922 - accuracy: 0.9036 - val_loss: 0.3146 - val_accuracy: 0.9055 - lr: 1.0000e-05
Epoch 3/5
599/599 [==============================] - 184s 306ms/step - loss: 0.1807 - accuracy: 0.9385 - val_loss: 0.2890 - val_accuracy: 0.9109 - lr: 1.0000e-05
Epoch 4/5
599/599 [==============================] - 184s 307ms/step - loss: 0.1216 - accuracy: 0.9579 - val_loss: 0.2719 - val_accuracy: 0.9166 - lr: 1.0000e-05
Epoch 5/5
599/599 [==============================] - 184s 307ms/step - loss: 0.0837 - accuracy: 0.9722 - val_loss: 0.2660 - val_accuracy: 0.9234 - lr: 1.0000e-05


In [7]:
new_model.save('arabic_letter_classifier_97_92')

In [6]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Dense


# Load the model
loaded_model2 = tf.keras.models.load_model("arabic_letter_classifier_95_87")

old_weights2 = loaded_model2.layers[-1].get_weights()



x = loaded_model2.layers[-2].output  # Target the last Dense layer before softmax
x = Dropout(0.3, name="new_dropout")(x)  # Add Dropout (rename it)
x = Dense(len(class_names), activation='softmax', name="new_output")(x)  # Rename new Dense layer

# Create new model (Keeps trained weights)
new_model2 = Model(inputs=loaded_model2.input, outputs=x)

# Set the pre-trained weights back
new_model2.layers[-1].set_weights(old_weights2)  # Restore softmax layer weights

# Recompile and continue training
new_model2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

new_model2.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               524544

In [9]:
for layer in loaded_model2.layers:
    if layer.name == 'dense':
        layer.traiable = True
    else:
        layer.trainable = False


base_model2 = new_model2.get_layer('resnet50')

# Unfreeze some layers of the base model
fine_tune_at = 100  # Unfreeze from this layer onwards

base_model2.trainable = True
for layer in base_model2.layers[:fine_tune_at]:
    layer.trainable = False


for layer in new_model2.layers:
    print(f"{layer.name}: {layer.trainable}")
print()

new_model2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

new_model2.summary()

input_2: False
tf.__operators__.getitem: False
tf.nn.bias_add: False
resnet50: True
global_average_pooling2d: False
dense: True
new_dropout: True
new_output: True

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)      

Total params: 24,128,961
Trainable params: 19,994,177
Non-trainable params: 4,134,784
_________________________________________________________________


In [11]:
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=2,
        min_lr=1e-8
    )
]

history = new_model2.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    callbacks=callbacks
)

Epoch 1/5
599/599 [==============================] - 223s 339ms/step - loss: 1.0032 - accuracy: 0.7397 - val_loss: 0.3891 - val_accuracy: 0.8799 - lr: 1.0000e-05
Epoch 2/5
599/599 [==============================] - 193s 322ms/step - loss: 0.3707 - accuracy: 0.8782 - val_loss: 0.3047 - val_accuracy: 0.9034 - lr: 1.0000e-05
Epoch 3/5
599/599 [==============================] - 188s 314ms/step - loss: 0.2413 - accuracy: 0.9199 - val_loss: 0.2764 - val_accuracy: 0.9159 - lr: 1.0000e-05
Epoch 4/5
599/599 [==============================] - 186s 311ms/step - loss: 0.1711 - accuracy: 0.9404 - val_loss: 0.2551 - val_accuracy: 0.9196 - lr: 1.0000e-05
Epoch 5/5
599/599 [==============================] - 187s 311ms/step - loss: 0.1235 - accuracy: 0.9565 - val_loss: 0.2497 - val_accuracy: 0.9246 - lr: 1.0000e-05


In [12]:
new_model2.save('arabic_letter_classifier_95_92')

In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Dense


# Load the model
loaded_model3 = tf.keras.models.load_model("arabic_letter_classifier_95_87")

old_weights3 = loaded_model3.layers[-1].get_weights()



x = loaded_model3.layers[-2].output  # Target the last Dense layer before softmax
x = Dropout(0.5, name="new_dropout")(x)  # Add Dropout (rename it)
x = Dense(len(class_names), activation='softmax', name="new_output")(x)  # Rename new Dense layer

# Create new model (Keeps trained weights)
new_model3 = Model(inputs=loaded_model3.input, outputs=x)

# Set the pre-trained weights back
new_model3.layers[-1].set_weights(old_weights3)  # Restore softmax layer weights

# Recompile and continue training
new_model3.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

new_model3.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               5245

In [9]:
for layer in loaded_model3.layers:
    if layer.name == 'dense':
        layer.traiable = True
    else:
        layer.trainable = False


base_model3 = new_model3.get_layer('resnet50')

# Unfreeze some layers of the base model
fine_tune_at = 100  # Unfreeze from this layer onwards

base_model3.trainable = True
for layer in base_model3.layers[:fine_tune_at]:
    layer.trainable = False


for layer in new_model3.layers:
    print(f"{layer.name}: {layer.trainable}")
print()

new_model3.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

new_model3.summary()

input_2: False
tf.__operators__.getitem: False
tf.nn.bias_add: False
resnet50: True
global_average_pooling2d: False
dense: True
new_dropout: True
new_output: True

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)    

In [10]:
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=2,
        min_lr=1e-8
    )
]

history = new_model3.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks
)

Epoch 1/10
599/599 [==============================] - 182s 300ms/step - loss: 1.5437 - accuracy: 0.6420 - val_loss: 0.4007 - val_accuracy: 0.8731 - lr: 1.0000e-05
Epoch 2/10
599/599 [==============================] - 327s 547ms/step - loss: 0.6134 - accuracy: 0.8035 - val_loss: 0.3018 - val_accuracy: 0.9048 - lr: 1.0000e-05
Epoch 3/10
599/599 [==============================] - 179s 298ms/step - loss: 0.4243 - accuracy: 0.8593 - val_loss: 0.2644 - val_accuracy: 0.9152 - lr: 1.0000e-05
Epoch 4/10
599/599 [==============================] - 190s 317ms/step - loss: 0.3166 - accuracy: 0.8936 - val_loss: 0.2340 - val_accuracy: 0.9276 - lr: 1.0000e-05
Epoch 5/10
599/599 [==============================] - 179s 298ms/step - loss: 0.2446 - accuracy: 0.9173 - val_loss: 0.2237 - val_accuracy: 0.9293 - lr: 1.0000e-05
Epoch 6/10
599/599 [==============================] - 179s 298ms/step - loss: 0.1949 - accuracy: 0.9339 - val_loss: 0.2161 - val_accuracy: 0.9354 - lr: 1.0000e-05
Epoch 7/10
599/599 [==

In [11]:
new_model3.save('arabic_letter_classifier_97_94')

In [13]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Dense


# Load the model
loaded_model4 = tf.keras.models.load_model("arabic_letter_classifier_95_87")

old_weights4 = loaded_model4.layers[-1].get_weights()



x = loaded_model4.layers[-2].output  # Target the last Dense layer before softmax
x = Dropout(0.7, name="new_dropout")(x)  # Add Dropout (rename it)
x = Dense(len(class_names), activation='softmax', name="new_output")(x)  # Rename new Dense layer

# Create new model (Keeps trained weights)
new_model4 = Model(inputs=loaded_model4.input, outputs=x)

# Set the pre-trained weights back
new_model4.layers[-1].set_weights(old_weights4)  # Restore softmax layer weights

# Recompile and continue training
new_model4.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

new_model4.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               5245

In [14]:
for layer in loaded_model4.layers:
    if layer.name == 'dense':
        layer.traiable = True
    else:
        layer.trainable = False


base_model4 = new_model4.get_layer('resnet50')

# Unfreeze some layers of the base model
fine_tune_at = 100  # Unfreeze from this layer onwards

base_model4.trainable = True
for layer in base_model4.layers[:fine_tune_at]:
    layer.trainable = False


for layer in new_model4.layers:
    print(f"{layer.name}: {layer.trainable}")
print()

new_model4.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

new_model4.summary()

input_2: False
tf.__operators__.getitem: False
tf.nn.bias_add: False
resnet50: True
global_average_pooling2d: False
dense: True
new_dropout: True
new_output: True

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)    

In [15]:
history = new_model4.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks
)

Epoch 1/10
599/599 [==============================] - 183s 301ms/step - loss: 2.6921 - accuracy: 0.4703 - val_loss: 0.4580 - val_accuracy: 0.8620 - lr: 1.0000e-05
Epoch 2/10
599/599 [==============================] - 176s 293ms/step - loss: 1.1544 - accuracy: 0.6507 - val_loss: 0.3540 - val_accuracy: 0.8971 - lr: 1.0000e-05
Epoch 3/10
599/599 [==============================] - 176s 294ms/step - loss: 0.8434 - accuracy: 0.7356 - val_loss: 0.2864 - val_accuracy: 0.9166 - lr: 1.0000e-05
Epoch 4/10
599/599 [==============================] - 176s 294ms/step - loss: 0.6663 - accuracy: 0.7853 - val_loss: 0.2472 - val_accuracy: 0.9241 - lr: 1.0000e-05
Epoch 5/10
599/599 [==============================] - 176s 294ms/step - loss: 0.5189 - accuracy: 0.8343 - val_loss: 0.2255 - val_accuracy: 0.9316 - lr: 1.0000e-05
Epoch 6/10
599/599 [==============================] - 191s 319ms/step - loss: 0.4238 - accuracy: 0.8637 - val_loss: 0.2126 - val_accuracy: 0.9349 - lr: 1.0000e-05
Epoch 7/10
599/599 [==

In [16]:
new_model4.optimizer.learning_rate.assign(1e-7)

history = new_model4.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    initial_epoch=10,
    callbacks=callbacks
)

Epoch 11/15
599/599 [==============================] - 177s 294ms/step - loss: 0.1663 - accuracy: 0.9454 - val_loss: 0.1930 - val_accuracy: 0.9450 - lr: 1.0000e-07
Epoch 12/15
599/599 [==============================] - 176s 294ms/step - loss: 0.1625 - accuracy: 0.9462 - val_loss: 0.1924 - val_accuracy: 0.9455 - lr: 1.0000e-07
Epoch 13/15
599/599 [==============================] - 176s 294ms/step - loss: 0.1564 - accuracy: 0.9484 - val_loss: 0.1918 - val_accuracy: 0.9457 - lr: 1.0000e-07
Epoch 14/15
599/599 [==============================] - 177s 295ms/step - loss: 0.1585 - accuracy: 0.9473 - val_loss: 0.1916 - val_accuracy: 0.9455 - lr: 1.0000e-07
Epoch 15/15
599/599 [==============================] - 176s 294ms/step - loss: 0.1569 - accuracy: 0.9485 - val_loss: 0.1914 - val_accuracy: 0.9452 - lr: 1.0000e-07


In [17]:
new_model4.save('arabic_letter_classifier_95_95')

In [18]:
new_model4.optimizer.learning_rate.assign(1e-9)

history = new_model4.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    initial_epoch=15,
    callbacks=callbacks
)

Epoch 16/20
599/599 [==============================] - 186s 308ms/step - loss: 0.1539 - accuracy: 0.9481 - val_loss: 0.1913 - val_accuracy: 0.9450 - lr: 1.0000e-09
Epoch 17/20
599/599 [==============================] - 179s 298ms/step - loss: 0.1550 - accuracy: 0.9496 - val_loss: 0.1913 - val_accuracy: 0.9452 - lr: 1.0000e-09
Epoch 18/20
599/599 [==============================] - 178s 297ms/step - loss: 0.1537 - accuracy: 0.9511 - val_loss: 0.1913 - val_accuracy: 0.9452 - lr: 1.0000e-09
Epoch 19/20
599/599 [==============================] - 177s 295ms/step - loss: 0.1521 - accuracy: 0.9486 - val_loss: 0.1912 - val_accuracy: 0.9452 - lr: 1.0000e-09
Epoch 20/20
599/599 [==============================] - 181s 301ms/step - loss: 0.1549 - accuracy: 0.9470 - val_loss: 0.1912 - val_accuracy: 0.9452 - lr: 1.0000e-09
